In [ ]:
# openai 라이브러리 확인
!pip show openai
# streamlit 라이브러리 확인
!pip show streamlit
# requests 라이브러리 확인
!pip show requests

2. 로고 디자인 생성 웹 애플리케이션 만들기

In [ ]:
%%writefile ./project22/streamlit_app.py

import streamlit as st
from openai import OpenAI
import requests
from PIL import Image
from io import BytesIO

# 1 로고 생성 generate_logo() 함수
def generate_logo(client, description):
    """DALL-E API를 사용하여 로고 이미지 생성"""
    prompt = f"Company Logo design: {description}"
    
    response = client.images.generate(
        model="dall-e-3",
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    return response.data[0].url, prompt
    
def main():
    # 2 앱 제목 설정
    st.title("🎨 AI 로고 생성기")
    
    # 3 API 키 입력 섹션
    api_key = st.text_input("OpenAI API 키를 입력하세요", type="password")
    
    # 4 사용자 입력 폼
    org_description = st.text_area("로고 설명",
        placeholder="예: 녹색과 파란색을 사용한 환경 친화적인 로고. 나뭇잎과 물방울 모양 선호.")
    file_name = st.text_input("저장할 파일명", placeholder="logo.png")
    
    # 5 로고 생성 버튼 및 처리
    if st.button("로고 생성하기") and api_key and org_description:
        with st.spinner("로고를 생성하는 중..."):
    
            try:
                # 6 OpenAI 클라이언트 초기화
                client = OpenAI(api_key=api_key)
                
                # 7 로고 생성 generate_logo( ) 함수로 이미지 생성
                image_url, prompt = generate_logo(client, org_description)
                
                # 8 사용된 프롬프트와 생성된 로고를 결과로 표시
                st.write("### 사용된 프롬프트")
                st.info(prompt)
                
                st.write("### 생성된 로고")
                st.image(image_url)
                
                # 9 다운로드 기능 구현
                response = requests.get(image_url)
                img = Image.open(BytesIO(response.content))
                img_byte_arr = BytesIO()
                img.save(img_byte_arr, format='PNG')
                img_byte_arr = img_byte_arr.getvalue()
                
                download_filename = file_name if file_name.endswith('.png') else f"{file_name}.png"
                st.download_button(
                    label="로고 다운로드",
                    data=img_byte_arr,
                    file_name=download_filename,
                    mime="image/png"
                )
            except Exception as e:
                st.error(f"오류가 발생했습니다: {str(e)}")
                
if __name__ == "__main__":
    main()

In [ ]:
# 스트림릿 앱 실행
!streamlit run ./project22/streamlit_app.py